In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
%%capture
# this is github gist URL, there are all functions and imports I need to reuse often across many notebooks
!wget -O package.py https://gist.githubusercontent.com/Jankoetf/c36cc24ddd83d4194148a86f87efd397/raw/1cedbb723936bf2dd1512b51ce9fee88edc7122e/package1.py
import package

In [28]:
'''import'''
dataset = pd.read_csv('jobfair_train.csv')
test_set = pd.read_csv('jobfair_test.csv')


'''sorting'''
dataset = dataset.sort_values(by='league_id')
clubs = test_set['club_id']
#dataset = dataset.sort_values(by='league_id')

'''preprocesing'''
dataset = package.basic_preprocessing(dataset)
dataset = package.Feature_Selection(dataset)
dataset = package.Averaging_by_leagues(dataset)

test_set = package.basic_preprocessing(test_set)
test_set = package.Feature_Selection(test_set)
test_set = package.Averaging_by_leagues(test_set)

print(dataset.shape)
print(test_set.shape)

(55314, 18)
(60270, 17)


# Prediction

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = package.train_val_test_split_adapted_shuffled(dataset)

In [6]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_val_scaled = sc.transform(X_val)

In [13]:
X_test_scaled = sc.transform(test_set.iloc[:,:].values)
print(X_test_scaled.shape)

(60270, 17)


In [7]:
best_model = Sequential()
best_model.add(Dense(10, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=l2(0.0001)))
best_model.add(Dense(10, activation='relu', kernel_regularizer=l2(0.0001)))
best_model.add(Dense(10, activation='relu', kernel_regularizer=l2(0.0001)))
best_model.add(Dense(1, activation='linear'))
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_absolute_error', metrics=['mae'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

best_model.fit(X_train_scaled, y_train, batch_size = 14, epochs = 25, \
        validation_data=(X_val_scaled, y_val), callbacks=[early_stopping])

**Postprocessing**

In [10]:
raw_predictions_test = best_model.predict(X_test_scaled)
predictions_test = package.post_processing_1(raw_predictions_test)

1884/1884 [==============================] - 3s 1ms/step


# Saving a results

In [29]:
test_set['league_rank'] = predictions_test
test_set['club_id'] = clubs

In [33]:
print(test_set.columns)

Index(['league_id', 'averaged_club_id', 'averaged_dynamic_payment_segment',
       'averaged_avg_age_top_11_players', 'averaged_avg_stars_top_11_players',
       'averaged_avg_training_factor_top_11_players',
       'averaged_days_active_last_28_days',
       'averaged_league_match_watched_count_last_28_days',
       'averaged_session_count_last_28_days', 'averaged_playtime_last_28_days',
       'averaged_league_match_won_count_last_28_days',
       'averaged_training_count_last_28_days',
       'averaged_global_competition_level',
       'averaged_tokens_spent_last_28_days', 'averaged_tokens_stash',
       'averaged_rests_stash', 'averaged_morale_boosters_stash', 'league_rank',
       'club_id'],
      dtype='object')


In [34]:
new_dataset = test_set[['club_id', 'league_rank']]
new_dataset.to_csv("league_rank_predictions.csv", index = False)

In [31]:
print(new_dataset.columns)

Index(['club_id', 'league_rank'], dtype='object')
